Import modules
===

In [1]:
# standard modules
import numpy as np
from IPython.display import Image, display
from os import mkdir
from os.path import isdir, isfile

In [2]:
# custom package

# load the measurement schemes from the Benchmark
from shadowgrouping.measurement_schemes import Shadow_Grouping, Brute_force_matching, Derandomization, AdaptiveShadows, L1_sampler,  hit_by
from shadowgrouping.measurement_schemes import SettingSampler as Overlapped_Grouping
from shadowgrouping.AEQuO import AEQuO
from shadowgrouping.weight_functions import Inconfidence_bound, Bernstein_bound
# wrapper class to combine the measurement scheme with the respective outcomes
from shadowgrouping.energy_estimator import Energy_estimator, StateSampler, Sign_estimator, save_energy_estimations
# helper functions to load Hamiltonian decompositions
from shadowgrouping.measurement_schemes import setting_to_str
from shadowgrouping.hamiltonian import get_pauli_list, get_groundstate, char_to_int, int_to_char, mappings, load_pauli_list
import shadowgrouping.molecules as molecules
from shadowgrouping.benchmark import benchmark_empirical, benchmark_provable, save_dict, save_dict_provable
# function to check pair-wise commutivity check
from shadowgrouping.Functions import is_commuting

print(molecules.available_molecules)

C:\Users\aebad\AppData\Local\Temp\ipykernel_21656\470293421.py:13: NatureDeprecationWarning: The qiskit_nature.drivers.Molecule class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.drivers.Molecule class.
  import shadowgrouping.molecules as molecules


ImportError: cannot import name 'is_commuting' from 'shadowgrouping.Functions' (C:\Users\aebad\shadowgrouping\shadowgrouping\Functions.py)

Data I/O preparation
===

In [ ]:
if not isdir("data/"):
    print("Data is zipped. Unzipping ...")
    !unzip -q data.zip
    print("Done!")

In [ ]:
folder_Hamiltonians = "Hamiltonians/"
folder_OGM_settings = "OGM_probabilities/OGM_{}_{}{}.txt" # format string to fill in {molecule}x{qubit_number}x{mapping}
savepath = "generated_data/"
savename = "_molecule_{}_empirical.txt" # insert {mapping_name}

Molecule specifics
---

In [ ]:
molecule_name = "H2" # choose one out of the molecules above
mapping_name = "JW" # choose one out of ["JW","BK","Parity"]
basis_set = "sto3g" # choose one out of ["sto3g","6-31g"] - the latter only for H2 molecule

Algorithm / Benchmark specifics
---

In [ ]:
eps = 0.1 # accuracy in Hartree
NUM_STEPS = 1000 # number of total measurement settings
N_runs = 100 # number of sampling repetitions for the same settings
N_plot = 30 # number of data points taken for fig4
delta = 0.02 # see caption of Figure 3 in manuscript
alpha_offset = 1e-3 # parameter for ShadowGrouping
UPPER_LIMIT_BRUTE_FORCE = 8 # in number of qubits

Ensure availability of folder
---

In [ ]:
savename = molecule_name + savename
# create temporary folder for storing outputs
if not isdir(savepath):
    mkdir(savepath)

Figure 2 - Truncation criterion
===
No data is used for generating this plot and no data is produced.

In [ ]:
!python3 demo_fig2.py

In [ ]:
display(Image(filename="generated_figures/fig2_demo.png"))

Table 1 - Empirical benchmark
===

In [ ]:
# observables, w, offset, E_GS, state = load_pauli_list(folder_Hamiltonians,molecule_name,basis_set,mapping_name,verbose=True)
observables, w, offset, _, _ = load_pauli_list(folder_Hamiltonians,molecule_name,basis_set,mapping_name,verbose=True,diagonalize=False)
# wrap ground state <state> into StateSampler in order to retrieve samples in arbitrary
state_sampler = StateSampler(state)
# fill in format string for Overlapped Grouping probabilities
folder_OGM_settings = folder_OGM_settings.format(molecule_name,observables.shape[-1],mapping_name.lower())

# hyperparameters for ShadowGrouping, see eq. (48) in manuscript
alpha = np.max(np.abs(w))/np.min(np.abs(w)) + alpha_offset
#q = np.array([3, 0, 0, 0])
#p = np.array([0, 3, 0, 0])
print(observables)

In [ ]:
# initialization of all methods used in the benchmark. The order corresponds to the one in Table I from left to right
methods = {}
methods["ShadowDerandomization"]     = Shadow_Grouping(observables,w,eps,Inconfidence_bound()())
methods["ShadowBernstein"]           = Shadow_Grouping(observables,w,eps,Bernstein_bound(alpha=alpha)())
methods["ShadowBernstein-truncated"] = Shadow_Grouping(observables,w,eps,Bernstein_bound(alpha=alpha)())
methods["Brute-force"]               = Brute_force_matching(observables,w,eps,Bernstein_bound(alpha=alpha)())
methods["Derandomization"]           = Derandomization(observables,w,eps,use_one_norm=True)
methods["RandomPaulis"]              = Derandomization(observables,w,eps,delta=1) # delta sets this method to random Paulis
methods["AdaptivePaulis"]            = AdaptiveShadows(observables,w)
methods["hit_by"]                    = hit_by(q,p)
methods["AEQuO"]                     = AEQuO(observables,w,offset,adaptiveness_L=2,interval_skewness_l=4,budget=NUM_STEPS) # values from figure 5 (L == adaptiveness_L +1)
# catch exception of missing data for OGM
file = folder_OGM_settings.format(molecule_name,observables.shape[1],mapping_name.lower())
if isfile(file):
    methods["OverlappedGrouping"]    = Overlapped_Grouping(observables,w,file)

Benchmarking all methods
---

In [ ]:
# all details can be found in benchmark.py
results = {}
for label,method in methods.items():
    params = {"Nshots":NUM_STEPS, "Nreps": N_runs}
    if label=="Brute-force":
        if observables.shape[1] > UPPER_LIMIT_BRUTE_FORCE:
            continue
    elif label=="ShadowBernstein-truncated":
        params["truncate_delta"] = delta # indicate to use the truncation criterion
    elif label=="AEQuO":
        params["use_naive"] = True # numerical necessity for compatability between AEQuO and the rest of the API
    rmse, std, _ = benchmark_empirical(method,offset,state_sampler,E_GS,params)
    results[label] = (rmse,std)
    print("Data for label <{}> generated.".format(label))

In [ ]:
# save data
# create temporary folder for storing outputs
if not isdir(savepath+"tab1/"):
    mkdir(savepath+"tab1/")
save_dict(savepath+"tab1/"+savename.format(mapping_name),results)
print("Empirical benchmark: data written to file")

Check outcomes
---
All previously generated data files can be easily accessed and displayed using the provided ```demo_*.py```-files
Using the ```-f <folder-name>``` option allows to look at the data generated and stored above.
If this option is left out, the data used for the manuscript is pulled up, instead

In [ ]:
!python3 demo_tab1.py -f generated_data/tab1/

Figure 4 - Provable benchmark
===

In [ ]:
savename = savename.replace("empirical","provable")
# this benchmark does not require knowledge about the GS or its energy
observables, w, offset, _, _ = load_pauli_list(folder_Hamiltonians,molecule_name,basis_set,mapping_name,verbose=True,diagonalize=False)
# fill in format string for Overlapped Grouping probabilities
folder_OGM_settings = folder_OGM_settings.format(molecule_name,observables.shape[-1],mapping_name.lower())

# hyperparameters for ShadowGrouping, see eq. (48) in manuscript
alpha = np.max(np.abs(w))/np.min(np.abs(w)) + alpha_offset

In [ ]:
# initialization of all methods used in the benchmark. The order corresponds to the one in Table I from left to right
methods = {}
methods["ShadowDerandomization"]     = Shadow_Grouping(observables,w,eps,Inconfidence_bound()())
methods["ShadowBernstein"]           = Shadow_Grouping(observables,w,eps,Bernstein_bound(alpha=alpha)())
# its truncated version is already included in the benchmark method
methods["AdaptivePaulis"]            = AdaptiveShadows(observables,w)
# catch exception of missing data for OGM
file = folder_OGM_settings.format(molecule_name,observables.shape[1],mapping_name.lower())
if isfile(file):
    methods["OverlappedGrouping"]    = Overlapped_Grouping(observables,w,file)

Benchmarking all methods
---

In [ ]:
# all details can be found in benchmark.py
results = {}
for label,method in methods.items():
    params = {"Nshots":NUM_STEPS, "Nreps": N_runs, "Nsteps": N_plot}
    if label=="ShadowBernstein":
        params["truncate"] = True # indicate to use the truncation criterion
        Nsteps, epsilons, epsilons_truncated = benchmark_provable(method,delta,params)
        results[label+"-truncated"] = epsilons_truncated
        print("Data for label <{}-truncated> generated.".format(label))
    else:
        Nsteps, epsilons = benchmark_provable(method,delta,params)
    results[label] = epsilons
    print("Data for label <{}> generated.".format(label))

In [ ]:
# save data
# create temporary folder for storing outputs
if not isdir(savepath+"fig4/"):
    mkdir(savepath+"fig4/")
save_dict_provable(savepath+"fig4/"+savename.format(mapping_name),results,Nsteps)
print("Provable benchmark: data written to file")

Check outcomes
---
All previously generated data files can be easily accessed and displayed using the provided ```demo_*.py```-files
Using the ```-f <folder-name>``` option allows to look at the data generated and stored above.
If this option is left out, the data used for the manuscript is pulled up, instead

In [ ]:
!python3 demo_fig4.py -f generated_data/fig4/

In [ ]:
display(Image(filename="generated_figures/fig4_demo_{}.png".format(molecule_name)))

Note: if you only see one of three subplots filled, run the above code for ```mapping_name = "JW", "BK", "Parity"```

Figure 5 - Ressource estimation
===
No data is used for generating this plot and no data is produced.

In [ ]:
!python3 demo_fig5.py

In [ ]:
display(Image(filename="generated_figures/fig5_demo.png"))

In [ ]:
def commute(p, q):
    """ Check if two observables commute """
    commutator = np.dot(p, q) - np.dot(q, p)
    return np.allclose(commutator, 0)  # Check if commutator is close to zero

# Call the function to check if observables commute
#commutes = commute(p, q)

# Print the result
print("Observables commute:", commutes)


In [ ]:
# Assuming you have the `commute` function defined as before
for i in range(len(observables)):
    for j in range(i + 1, len(observables)):
        result = commute(observables[i], observables[j])
        print(f"Observables {i} and {j} commute:", result)

In [ ]:
# Assuming you have the `hit_by` function defined as before
for i in range(len(observables)):
    for j in range(i + 1, len(observables)):
        result = hit_by(observables[i], observables[j])
        print(f"Observables {i} and {j} commute:", result)